In [1]:
from IPython.core.interactiveshell import InteractiveShell  #---- Output all jupyter lab inputs instead of the last one
from IPython.display import Markdown, display
InteractiveShell.ast_node_interactivity = "all"             
import glob #---------------------------------------------------- To read the files or folders in a system directory
from netCDF4 import Dataset #------------------------------------ To read nc , nc4 and hdf4 files
import numpy as np
import datetime
import h5py #---------------------------------------------------- To read hdf5 files
from scipy import spatial #-------------------------------------- To extract the values and indices of k nearest neighbors
import pandas as pd
from ast import literal_eval #----------------------------------- For literal evaluation of a string to extract python objects
from pyproj import Proj, transform #----------------------------- To interconvert different projections
import warnings #------------------------------------------------ To suppress warnings
from photutils.utils import ShepardIDWInterpolator as idw #------ To use Shepard's Inverse Distance Weighing Interpolation tool
import re #------------------------------------------------------ To replace characters in a string
warnings.simplefilter('ignore')
def printmd(string):
    display(Markdown(string))

In [74]:
#sif_list  = sorted(glob.glob('Processed_sif/*.csv' ))
#par_list  = sorted(glob.glob('Processed_par/*.csv' ))
#ref_list  = sorted(glob.glob('Processed_ref/*.csv' ))
#fpar_list = sorted(glob.glob('Processed_fpar/*.csv'))

#for i_sif, i_par, i_ref, i_fpar in zip(sif_list, par_list, ref_list, fpar_list):

di  = pd.read_csv('df_sif_2018-05-03.csv' )
dj  = pd.read_csv('df_par_2018-05-03.csv' )
dk  = pd.read_csv('df_ref_2018-05-03.csv' )
dl  = pd.read_csv('df_fpar_2018-05-03.csv')
dM1 = di .merge(dj, on=['sounding_id','latitude','longitude'], how='inner')
dM2 = dM1.merge(dk, on=['sounding_id','latitude','longitude'], how='inner')
dM3 = dM2.merge(dl, on=['sounding_id','latitude','longitude'], how='inner')

dM3 = dM3.dropna(subset=['par'])#'Fpar_500m','Lai_500m'])
#dM3  = dM3.copy()
L   = len(dM3)
print('length:',L)
#dM3.to_csv('APPLE.csv',index=False)
#dM3.info()
#dM3.sounding_id[:5]

file          = Dataset('New_Data_just_empty_par_dropped_2018-05-03.nc4', 'w', format='NETCDF4')
group_sif     = file.createGroup("Group_OCO2"       )
group_par     = file.createGroup("Group_MCD18A2"    )
group_ref     = file.createGroup("Group_MCD43A4"    )
group_fpar    = file.createGroup("Group_MCD15A3H"   )

vvSIF_757nm                          = file.createDimension('SIF_757nm', L)
vvcloud_albedo                       = file.createDimension('cloud_albedo', L)
vvcloud_flag                         = file.createDimension('cloud_flag', L)
vvcloud_co2_ratio                    = file.createDimension('cloud_co2_ratio', L)
vvcloud_delta_surface_pressure       = file.createDimension('cloud_delta_surface_pressure', L)
vvcloud_o2_ratio                     = file.createDimension('cloud_o2_ratio', L)
vvvapor_pressure_deficit             = file.createDimension('vapor_pressure_deficit', L)
vvtemperature_2m                     = file.createDimension('temperature_2m', L)
vvtemperature_skin                   = file.createDimension('temperature_skin', L)
vvspecific_humidity                  = file.createDimension('specific_humidity', L)
vvsurface_pressure                   = file.createDimension('surface_pressure',L)
vvwind_speed                         = file.createDimension('wind_speed',L)
vvcontinuum_radiance_757nm           = file.createDimension('continuum_radiance_757nm',L)
vvcontinuum_radiance_771nm           = file.createDimension('continuum_radiance_771nm',L)
vvdaily_correction_factor            = file.createDimension('daily_correction_factor',L)
vvfootprint                          = file.createDimension('footprint',L)
vvIGBP_index                         = file.createDimension('IGBP_index',L)
vvlatitude                           = file.createDimension('latitude',L)
vvlongitude                          = file.createDimension('longitude',L)
vvmeasurement_mode                   = file.createDimension('measurement_mode',L)
vvorbit_number                       = file.createDimension('orbit_number',L)
vvreduced_chi2_757nm                 = file.createDimension('reduced_chi2_757nm',L)
vvreduced_chi2_771nm                 = file.createDimension('reduced_chi2_771nm',L)
vvsensor_azimuth_angle               = file.createDimension('sensor_azimuth_angle',L)
vvsensor_zenith_angle                = file.createDimension('sensor_zenith_angle',L)
vvSIF_757nm_relative                 = file.createDimension('SIF_757nm_relative',L)
vvSIF_757nm_uncert                   = file.createDimension('SIF_757nm_uncert',L)
vvSIF_771nm                          = file.createDimension('SIF_771nm',L)
vvSIF_771nm_relative                 = file.createDimension('SIF_771nm_relative',L)
vvSIF_771nm_uncert                   = file.createDimension('SIF_771nm_uncert',L)
vvsolar_azimuth_angle                = file.createDimension('solar_azimuth_angle',L)
vvsolar_zenith_angle                 = file.createDimension('solar_zenith_angle',L)
vvsounding_id                        = file.createDimension('sounding_id',L)
vvsurface_altitude                   = file.createDimension('surface_altitude',L)
vvtime                               = file.createDimension('time',L)
vvuncorrected_SIF_757nm              = file.createDimension('uncorrected_SIF_757nm',L)
vvuncorrected_SIF_757nm_relative     = file.createDimension('uncorrected_SIF_757nm_relative',L)
vvuncorrected_SIF_771nm              = file.createDimension('uncorrected_SIF_771nm',L)
vvuncorrected_SIF_771nm_relative     = file.createDimension('uncorrected_SIF_771nm_relative',L)
vvPAR                                = file.createDimension('PAR' ,L)
vvNRB1                               = file.createDimension('Nadir_Reflectance_Band1',L)
vvNRB2                               = file.createDimension('Nadir_Reflectance_Band2',L)
vvfpar500                            = file.createDimension('Fpar_500m',L)
vvlai500                             = file.createDimension('Lai_500m',L)

vvSIF_757nm                         = file.createVariable('/Group_OCO2/SIF_757nm',float, ('SIF_757nm',))
vvcloud_albedo                      = file.createVariable('/Group_OCO2/cloud_albedo', float, ('cloud_albedo',))
vvcloud_flag                        = file.createVariable('/Group_OCO2/cloud_flag', float, ('cloud_flag',))
vvcloud_co2_ratio                   = file.createVariable('/Group_OCO2/cloud_co2_ratio', float,('cloud_co2_ratio',))
vvcloud_delta_surface_pressure      = file.createVariable('/Group_OCO2/cloud_delta_surface_pressure', float,('cloud_delta_surface_pressure',))
vvcloud_o2_ratio                    = file.createVariable('/Group_OCO2/cloud_o2_ratio', float,('cloud_o2_ratio',))
vvvapor_pressure_deficit            = file.createVariable('/Group_OCO2/vapor_pressure_deficit', float,('vapor_pressure_deficit',))
vvtemperature_2m                    = file.createVariable('/Group_OCO2/temperature_2m', float,('temperature_2m',))
vvtemperature_skin                  = file.createVariable('/Group_OCO2/temperature_skin', float,('temperature_skin',))
vvspecific_humidity                 = file.createVariable('/Group_OCO2/specific_humidity', float,('specific_humidity',))
vvsurface_pressure                  = file.createVariable('/Group_OCO2/surface_pressure',float,('surface_pressure',))
vvwind_speed                        = file.createVariable('/Group_OCO2/wind_speed',float,('wind_speed',))
vvcontinuum_radiance_757nm          = file.createVariable('/Group_OCO2/continuum_radiance_757nm',float,('continuum_radiance_757nm',))
vvcontinuum_radiance_771nm          = file.createVariable('/Group_OCO2/continuum_radiance_771nm',float,('continuum_radiance_771nm',))
vvdaily_correction_factor           = file.createVariable('/Group_OCO2/daily_correction_factor',float,('daily_correction_factor',))
vvfootprint                         = file.createVariable('/Group_OCO2/footprint',float,('footprint',))
vvIGBP_index                        = file.createVariable('/Group_OCO2/IGBP_index',float,('IGBP_index',))
vvlatitude                          = file.createVariable('latitude',float,('latitude',),fill_value=0)
vvlongitude                         = file.createVariable('longitude',float,('longitude',),fill_value=0)
vvmeasurement_mode                  = file.createVariable('/Group_OCO2/measurement_mode',float,('measurement_mode',))
vvorbit_number                      = file.createVariable('/Group_OCO2/orbit_number',float,('orbit_number',))
vvreduced_chi2_757nm                = file.createVariable('/Group_OCO2/reduced_chi2_757nm',float,('reduced_chi2_757nm',))
vvreduced_chi2_771nm                = file.createVariable('/Group_OCO2/reduced_chi2_771nm',float,('reduced_chi2_771nm',))
vvsensor_azimuth_angle              = file.createVariable('/Group_OCO2/sensor_azimuth_angle',float,('sensor_azimuth_angle',))
vvsensor_zenith_angle               = file.createVariable('/Group_OCO2/sensor_zenith_angle',float,('sensor_zenith_angle',))
vvSIF_757nm_relative                = file.createVariable('/Group_OCO2/SIF_757nm_relative',float,('SIF_757nm_relative',))
vvSIF_757nm_uncert                  = file.createVariable('/Group_OCO2/SIF_757nm_uncert',float,('SIF_757nm_uncert',))
vvSIF_771nm                         = file.createVariable('/Group_OCO2/SIF_771nm',float,('SIF_771nm',))
vvSIF_771nm_relative                = file.createVariable('/Group_OCO2/SIF_771nm_relative',float,('SIF_771nm_relative',))
vvSIF_771nm_uncert                  = file.createVariable('/Group_OCO2/SIF_771nm_uncert',float,('SIF_771nm_uncert',))
vvsolar_azimuth_angle               = file.createVariable('/Group_OCO2/solar_azimuth_angle',float,('solar_azimuth_angle',))
vvsolar_zenith_angle                = file.createVariable('/Group_OCO2/solar_zenith_angle',float,('solar_zenith_angle',))
vvsounding_id                       = file.createVariable('sounding_id',float,('sounding_id',),fill_value=-999)
vvsurface_altitude                  = file.createVariable('/Group_OCO2/surface_altitude',float,('surface_altitude',))
vvtime                              = file.createVariable('time',float,('time',),fill_value=-9999)
vvuncorrected_SIF_757nm             = file.createVariable('/Group_OCO2/uncorrected_SIF_757nm',float,('uncorrected_SIF_757nm',))
vvuncorrected_SIF_757nm_relative    = file.createVariable('/Group_OCO2/uncorrected_SIF_757nm_relative',float,('uncorrected_SIF_757nm_relative',))
vvuncorrected_SIF_771nm             = file.createVariable('/Group_OCO2/uncorrected_SIF_771nm',float,('uncorrected_SIF_771nm',))
vvuncorrected_SIF_771nm_relative    = file.createVariable('/Group_OCO2/uncorrected_SIF_771nm_relative',float,('uncorrected_SIF_771nm_relative',))
vvPAR                               = file.createVariable('/Group_MCD18A2/PAR' ,float,('PAR' ,))
vvNRB1                              = file.createVariable('/Group_MCD43A4/Nadir_Reflectance_Band1',float,('Nadir_Reflectance_Band1',))
vvNRB2                              = file.createVariable('/Group_MCD43A4/Nadir_Reflectance_Band2',float,('Nadir_Reflectance_Band2',))
vvfpar500                           = file.createVariable('/Group_MCD15A3H/Fpar_500m',float,('Fpar_500m',))
vvlai500                            = file.createVariable('/Group_MCD15A3H/Lai_500m' ,float,('Lai_500m',))


vvSIF_757nm[:]                      = np.array(dM3['SIF_757nm'],dtype=float)
vvcloud_albedo[:]                   = np.array(dM3['cloud_albedo'])
vvcloud_flag[:]                     = np.array(dM3['cloud_flag'])
vvcloud_co2_ratio[:]                = np.array(dM3['cloud_co2_ratio'])
vvcloud_delta_surface_pressure[:]   = np.array(dM3['cloud_delta_surface_pressure'])
vvcloud_o2_ratio[:]                 = np.array(dM3['cloud_o2_ratio'])
vvvapor_pressure_deficit[:]         = np.array(dM3['vapor_pressure_deficit'])
vvtemperature_2m[:]                 = np.array(dM3['temperature_2m'])
vvtemperature_skin[:]               = np.array(dM3['temperature_skin'])
vvspecific_humidity[:]              = np.array(dM3['specific_humidity'])
vvsurface_pressure[:]               = np.array(dM3['surface_pressure'])
vvwind_speed[:]                     = np.array(dM3['wind_speed'])
vvcontinuum_radiance_757nm[:]       = np.array(dM3['continuum_radiance_757nm'])
vvcontinuum_radiance_771nm[:]       = np.array(dM3['continuum_radiance_771nm'])
vvdaily_correction_factor[:]        = np.array(dM3['daily_correction_factor'])
vvfootprint[:]                      = np.array(dM3['footprint'])
vvIGBP_index[:]                     = np.array(dM3['IGBP_index'])
vvlatitude[:]                       = np.array(dM3['latitude'])
vvlongitude[:]                      = np.array(dM3['longitude'])
vvmeasurement_mode[:]               = np.array(dM3['measurement_mode'])
vvorbit_number[:]                   = np.array(dM3['orbit_number'])
vvreduced_chi2_757nm[:]             = np.array(dM3['reduced_chi2_757nm'])
vvreduced_chi2_771nm[:]             = np.array(dM3['reduced_chi2_771nm'])
vvsensor_azimuth_angle[:]           = np.array(dM3['sensor_azimuth_angle'])
vvsensor_zenith_angle[:]            = np.array(dM3['sensor_zenith_angle'])
vvSIF_757nm_relative[:]             = np.array(dM3['SIF_757nm_relative'])
vvSIF_757nm_uncert[:]               = np.array(dM3['SIF_757nm_uncert'])
vvSIF_771nm[:]                      = np.array(dM3['SIF_771nm'])
vvSIF_771nm_relative[:]             = np.array(dM3['SIF_771nm_relative'])
vvSIF_771nm_uncert[:]               = np.array(dM3['SIF_771nm_uncert'])
vvsolar_azimuth_angle[:]            = np.array(dM3['solar_azimuth_angle'])
vvsolar_zenith_angle[:]             = np.array(dM3['solar_zenith_angle'])
vvsounding_id[:]                    = np.array(dM3['sounding_id'])
vvsurface_altitude[:]               = np.array(dM3['surface_altitude'])
vvtime[:]                           = np.array(dM3['time'])
vvuncorrected_SIF_757nm[:]          = np.array(dM3['uncorrected_SIF_757nm'])
vvuncorrected_SIF_757nm_relative[:] = np.array(dM3['uncorrected_SIF_757nm_relative'])
vvuncorrected_SIF_771nm[:]          = np.array(dM3['uncorrected_SIF_771nm'])
vvuncorrected_SIF_771nm_relative[:] = np.array(dM3['uncorrected_SIF_771nm_relative'])
vvPAR [:]                           = np.array(dM3['par'])
vvNRB1[:]                           = np.array(dM3['nrb1'])
vvNRB2[:]                           = np.array(dM3['nrb2'])
vvfpar500[:]                        = np.array(dM3['Fpar_500m'])
vvlai500[:]                         = np.array(dM3['Lai_500m'])
file.close()


length: 3865


In [57]:
dM3

,SIF_757nm,cloud_albedo,cloud_flag,cloud_co2_ratio,cloud_delta_surface_pressure,cloud_o2_ratio,vapor_pressure_deficit,temperature_2m,temperature_skin,specific_humidity,...,Xdistance,Date,SIF_Time,tile_h,tile_v,par,nrb1,nrb2,Fpar_500m,Lai_500m
0,0.013955,0.196045,0.0,1.021912,-1197.379395,0.991028,2811.565430,300.584961,300.404297,0.005310,...,1.998464,2018-05-01,05:20:2002,27,12,NaN,NaN,NaN,NaN,NaN
1,0.162872,0.131165,0.0,1.026367,-1196.760010,0.999390,2888.075684,300.938049,300.975220,0.005310,...,1.477508,2018-05-01,05:20:2072,27,12,NaN,NaN,NaN,NaN,NaN
2,0.702497,0.148193,0.0,1.022400,-1196.450073,1.007690,2927.572266,301.117737,301.260376,0.005310,...,1.484524,2018-05-01,05:20:2102,27,12,NaN,NaN,NaN,NaN,NaN
3,0.001342,0.162048,0.0,1.027527,-1196.140259,0.998291,2966.201904,301.291016,301.545166,0.005310,...,0.979888,2018-05-01,05:20:2132,27,12,NaN,NaN,NaN,NaN,NaN
4,0.018588,0.215881,1.0,1.046631,-1194.898926,0.960693,3103.490234,301.891357,302.572632,0.005249,...,0.471414,2018-05-01,05:20:2272,27,12,NaN,NaN,NaN,NaN,NaN
5,0.001895,0.179077,1.0,1.153870,-1193.036865,0.855408,3189.326416,302.255615,303.231384,0.005249,...,1.737778,2018-05-01,05:20:2472,27,12,NaN,NaN,NaN,NaN,NaN
6,0.170996,0.257874,1.0,1.070862,-1191.488770,0.928772,3234.548828,302.438354,303.744263,0.005249,...,0.611889,2018-05-01,05:20:2632,27,12,NaN,NaN,NaN,NaN,NaN
7,0.010268,0.248474,1.0,1.121399,-1191.178711,0.892517,3241.893066,302.467407,303.843140,0.005249,...,1.890870,2018-05-01,05:20:2672,27,12,NaN,NaN,NaN,NaN,NaN
8,0.263684,0.247498,1.0,1.146606,-1190.868652,0.860535,3256.149658,302.527588,303.940857,0.005249,...,1.039403,2018-05-01,05:20:2702,27,12,NaN,NaN,NaN,NaN,NaN
9,0.267760,0.234009,1.0,1.096436,-1190.558472,0.920715,3269.485840,302.583618,304.037231,0.005249,...,1.400236,2018-05-01,05:20:2732,27,12,NaN,NaN,NaN,NaN,NaN


In [35]:
ff = Dataset('DATAs_123.nc4','r')
s = ff.groups['Group_OCO2'].variables['SIF_757nm'][:]
t = ff.variables['longitude'][:]
u = ff.variables['sounding_id'][:]
pd.Series(s).nunique()
pd.Series(t).nunique()
pd.Series(u).nunique()
s
t
u

3113

3113

3113

masked_array(data=[0.36817616, 0.48211259, 0.37076208, ..., 1.52201772,
                   0.17384332, 1.0403446 ],
             mask=False,
       fill_value=1e+20)

masked_array(data=[ 128.70755005,  128.6864624 ,  128.65493774, ...,
                   -113.78807068, -113.90844727, -113.92276001],
             mask=False,
       fill_value=1e+20)

masked_array(data=[2.01805011e+15, 2.01805011e+15, 2.01805011e+15, ...,
                   2.01805012e+15, 2.01805012e+15, 2.01805012e+15],
             mask=False,
       fill_value=1e+20)